In [1]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# LLMChain使用memory
- 之前的对话内容会作为上下文翻入prompt中

In [3]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

#自定义模板
template = """你是一个可以和人类对话的机器人.
{chat_history}
人类:{human_input}
机器人:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["chat_history", "human_input"],
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(temperature=0)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [4]:
chain.predict(human_input="你好")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个可以和人类对话的机器人.

人类:你好
机器人:

> Finished chain.


'你好，我是一个可以和人类对话的机器人。你有什么问题想要问我吗？'

In [5]:
chain.predict(human_input="我想去香港旅游")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个可以和人类对话的机器人.
Human: 你好
AI: 你好，我是一个可以和人类对话的机器人。你有什么问题想要问我吗？
人类:我想去香港旅游
机器人:

> Finished chain.


' 香港是一个非常美丽和繁华的城市，有很多著名的景点和美食。你可以去看看维多利亚港、太平山顶、迪士尼乐园等地方。不过，最近香港有些社会动荡，你需要提前做好安全措施。'

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            role="system",
            content="你好，我是一个可以和人类对话的机器人",

        ),
        MessagesPlaceholder(
            variable_name="chat_history",
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

print(prompt.format(human_input="你好", chat_history=[]))

System: 你好，我是一个可以和人类对话的机器人
Human: 你好


In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, )
llm = ChatOpenAI(temperature=0)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [10]:
chain.predict(human_input="我叫tomie，我是一个AI应用程序猿")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你好，我是一个可以和人类对话的机器人
Human: 我叫tomie，我是一个AI应用程序猿

> Finished chain.


'很高兴认识你，Tomie！作为一个AI应用程序猿，你对编程和人工智能一定有很深的了解。有什么关于编程或者人工智能方面的话题想和我讨论吗？'

# ConversationChain
- 简化了上述LLMChain中的对话存储memory形式

In [11]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

llm = OpenAI(
    temperature=0,
)
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)

In [12]:
chain.predict(input="你好,我想去北京玩")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: 你好,我想去北京玩
AI:

> Finished chain.


' 你好！欢迎来到北京！北京是中国的首都，也是一个历史悠久的城市。它有许多著名的景点，比如故宫、天安门广场、长城等等。你想去哪里玩呢？我可以为你提供详细的旅游建议。'

In [13]:
chain.predict(input="帮我做个一日游攻略")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='你好,我想去北京玩'), AIMessage(content=' 你好！欢迎来到北京！北京是中国的首都，也是一个历史悠久的城市。它有许多著名的景点，比如故宫、天安门广场、长城等等。你想去哪里玩呢？我可以为你提供详细的旅游建议。')]
Human: 帮我做个一日游攻略
AI:

> Finished chain.


' 当然可以！一日游攻略的话，我推荐你先去故宫，这是中国古代皇宫的代表，里面有许多珍贵的文物和建筑。然后可以去天安门广场，这是中国的政治中心，也是一个重要的历史纪念地。如果你喜欢自然风光，可以去颐和园，那里有美丽的湖泊和园林。最后，如果时间允许，可以去长城，这是中国的标志性建筑，也是世界上最长的防御工程。你觉得怎么样？'

# 自定义一下，覆盖ConversationChain
- 默认ConversationChain中格式比较固定(比如human,AI),我们可以自定义对话角色

In [14]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类brother，我真的不知道'.
当前对话:
{history}
Human:{input}
大头AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["history", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(
        ai_prefix="大头AI",
        return_messages=True,
    ),
    prompt=prompt,
    verbose=True,
)

In [15]:
chain.predict(input="你知道我叫什么名字吗?")



> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类brother，我真的不知道'.
当前对话:
[]
Human:你知道我叫什么名字吗?
大头AI:

> Finished chain.


' 人类brother，我真的不知道。'

# 同一个链合并使用多个memory

In [20]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import (ConversationBufferMemory, ConversationSummaryMemory, CombinedMemory)
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

#使用 ConversationSummaryMemory 对对话进行总结
summary_memory = ConversationSummaryMemory(llm=llm, input_key="input", memory_key="history_summary")
#使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(memory_key="history_now", input_key="input", )
# 组合使用
memory = CombinedMemory(memories=[summary_memory, cov_memory])

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
---------------
{history_summary}
---------------
当前对话:
---------------
{history_now}
---------------
Human:{input}
AI："""

prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["history_summary", "history_now", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [21]:
chain.invoke("你了解mq吗？")

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
---------------

---------------
当前对话:
---------------

---------------
Human:你了解mq吗？
AI：

> Finished chain.


'是的，MQ是一种消息队列，它可以帮助不同的应用程序之间进行通信和数据交换。它可以提高系统的可靠性和可扩展性，并且可以处理大量的消息。'

In [22]:
# 可以看到 我们使用了2种类型的memory在prompt中
chain.invoke("kafka怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
---------------

The human asks if the AI is familiar with MQ. The AI responds that MQ is a type of message queue that helps different applications communicate and exchange data. It also improves system reliability and scalability, and can handle large volumes of messages.
---------------
当前对话:
---------------
Human: 你了解mq吗？
AI: 是的，MQ是一种消息队列，它可以帮助不同的应用程序之间进行通信和数据交换。它可以提高系统的可靠性和可扩展性，并且可以处理大量的消息。
---------------
Human:kafka怎么样？
AI：

> Finished chain.


{'input': 'kafka怎么样？',
 'history_summary': '\nThe human asks if the AI is familiar with MQ. The AI responds that MQ is a type of message queue that helps different applications communicate and exchange data. It also improves system reliability and scalability, and can handle large volumes of messages.',
 'history_now': 'Human: 你了解mq吗？\nAI: 是的，MQ是一种消息队列，它可以帮助不同的应用程序之间进行通信和数据交换。它可以提高系统的可靠性和可扩展性，并且可以处理大量的消息。',
 'response': 'Kafka是一种分布式流处理平台，它可以帮助处理大量的实时数据流。它具有高吞吐量、低延迟和可扩展性的特点，被广泛应用于大数据领域。'}

# 多参数链增加记忆  

<hr>

In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

with open("testfile/letter.txt") as f:
    text = f.read()
    #切分文本
    text_splitter = CharacterTextSplitter(
        chunk_size=20,
        chunk_overlap=5
    )
    texts = text_splitter.split_text(text)
    #使用chroma向量存储
    docssearch = Chroma.from_texts(
        texts,
        #使用openai的embedding
        OpenAIEmbeddings(),
    )
    query = "公司有什么新策略?"
    docs = docssearch.similarity_search(query=query)

Created a chunk of size 24, which is longer than the specified 20
Created a chunk of size 45, which is longer than the specified 20
Created a chunk of size 50, which is longer than the specified 20
Created a chunk of size 232, which is longer than the specified 20
Created a chunk of size 589, which is longer than the specified 20
Created a chunk of size 610, which is longer than the specified 20
Created a chunk of size 561, which is longer than the specified 20
Created a chunk of size 563, which is longer than the specified 20
Created a chunk of size 259, which is longer than the specified 20
Created a chunk of size 87, which is longer than the specified 20
/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in t

In [24]:
#构建问答对话链
from langchain.chains.question_answering import  load_qa_chain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，
如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，
参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
{context}
{chat_history}
Human:{human_input}
AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "chat_history", "human_input"],
)

#使用ConversationBufferMemory对对话进行缓存 
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    return_messages=True,
)

#加载对话链
chain = load_qa_chain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
    chain_type="stuff"
)

chain({"input_documents":docs,"human_input":"公司的营销策略是什么？"})

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，
如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，
参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
Marketing Initiatives and Campaigns
Our marketing team has been actively working on developing new strategies to increase brand awareness and drive customer engagement. We would like to thank Sarah Thompson (phone: 415-555-1234) for her exceptional efforts in managing our social media platforms. Sarah has successfully increased our follower base by 20% in the past month alone. Moreover, please mark your calendars for the upcoming product launch event on July 15th. We encourage all team members to attend and support this exciting milestone for our company.

Dear Team,

Subject: Updates and Discussions on Various Topics

I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please

{'input_documents': [Document(page_content='Marketing Initiatives and Campaigns\nOur marketing team has been actively working on developing new strategies to increase brand awareness and drive customer engagement. We would like to thank Sarah Thompson (phone: 415-555-1234) for her exceptional efforts in managing our social media platforms. Sarah has successfully increased our follower base by 20% in the past month alone. Moreover, please mark your calendars for the upcoming product launch event on July 15th. We encourage all team members to attend and support this exciting milestone for our company.'),
  Document(page_content='Dear Team,'),
  Document(page_content='Subject: Updates and Discussions on Various Topics'),
  Document(page_content='I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please treat the information contained herein as highly confidential.')],
 'human_in

In [25]:
chain({"input_documents":docs,"human_input":"公司叫什么名称?"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，
如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，
参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
Marketing Initiatives and Campaigns
Our marketing team has been actively working on developing new strategies to increase brand awareness and drive customer engagement. We would like to thank Sarah Thompson (phone: 415-555-1234) for her exceptional efforts in managing our social media platforms. Sarah has successfully increased our follower base by 20% in the past month alone. Moreover, please mark your calendars for the upcoming product launch event on July 15th. We encourage all team members to attend and support this exciting milestone for our company.

Dear Team,

Subject: Updates and Discussions on Various Topics

I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please

{'input_documents': [Document(page_content='Marketing Initiatives and Campaigns\nOur marketing team has been actively working on developing new strategies to increase brand awareness and drive customer engagement. We would like to thank Sarah Thompson (phone: 415-555-1234) for her exceptional efforts in managing our social media platforms. Sarah has successfully increased our follower base by 20% in the past month alone. Moreover, please mark your calendars for the upcoming product launch event on July 15th. We encourage all team members to attend and support this exciting milestone for our company.'),
  Document(page_content='Dear Team,'),
  Document(page_content='Subject: Updates and Discussions on Various Topics'),
  Document(page_content='I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please treat the information contained herein as highly confidential.')],
 'human_in